### Combine the models using three different methods and compare them

In [10]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from cvxopt import matrix, solvers

In [11]:
def convert_Qc(Q, c_v):
    n = Q.shape[0]-1
    c = Q[-1,-1]
    t = Q[:n, -1:]
    Q = Q[:n,:n]
    c_v = c_v.reshape((-1,1))
    newQ = Q - np.dot(t, np.ones((1,n))) - np.transpose(np.dot(t, np.ones((1,n)))) + c*np.ones((n,n))
    newC = c_v[:n,-1:] + 2*t - (2*c + c_v[-1,-1])*np.ones((n,1))
    return (newQ + newQ.T)/2, newC

In [12]:
def mape(y_pred, y_true):
    diff = np.abs((y_true - y_pred) / np.clip(np.abs(y_true),
                                            1e-7,
                                            None))
    return 100. * np.mean(diff)

import properscoring as ps
def crps(mu, sig, y_true):
    crps = np.zeros((mu.shape[0], 1))
    for i in range(mu.shape[0]):
        crps[i] = ps.crps_gaussian(y_true[i], mu[i], sig[i])
    return np.mean(crps)

In [13]:
region = 'VT'

In [14]:
mu = np.zeros((840,15))
sigma  = np.zeros((840,15))

result_for_df = np.zeros((2,3))
for i in range(5):
    filename = 'results/GPR_'+region+'_result' + str(i+1) + '.csv'
    result = pd.read_csv(filename)
    mu[:,i] = result['mean'].values
    sigma[:,i] = result['sigma'].values
for i in range(5):
    filename = 'results/NN_'+region+'_result' + str(i+1) + '.csv'
    result = pd.read_csv(filename)
    mu[:,5+i] = result['mean'].values
    sigma[:,5+i] = result['std'].values
for i in range(5):
    filename = 'results/XGB_'+region+'_result' + str(i+1) + '.csv'
    result = pd.read_csv(filename)
    mu[:,10+i] = result['mean'].values
    sigma[:,10+i] = result['std'].values
    
y = pd.read_csv('Data/'+region+'_dev2.csv', usecols = ['DEMAND']).values.reshape(-1)    
mu = mu.T
sigma = sigma.T

n = mu.shape[0]   # number of models to combine
t = mu.shape[1] # number of data points
alpha = np.zeros((n, n, t))
beta = np.zeros((n, t))
for i in range(n):
    for j in range(n):
        sig = np.sqrt(sigma[i,:]*sigma[i,:] + sigma[j,:]*sigma[j,:])
        x = (mu[i,:] - mu[j,:]) / sig # shape (t,)
        alpha[i,j,:] = -1. * norm.pdf(x) * sig + -0.5 * (mu[i,:] - mu[j,:]) * (2. * norm.cdf(x) - 1.)
    xc = (mu[i,:] - y) / sigma[i,:]
    beta[i,:] = 2. * sigma[i,:] * norm.pdf(xc) + (mu[i,:] - y) * (2. * norm.cdf(xc) - 1.)

Q = np.mean(alpha, axis = -1)
c = np.mean(beta, axis = -1)
newQ, newC = convert_Qc(Q, c)

Q = 2*matrix(newQ)
c = matrix(newC)
G = matrix(0.0, (n,n-1))
G[::n+1] = -1.0
G[-1,:] = 1.0
h = matrix(0.0, (n,1))
h[-1,-1] = 1.0
sol=solvers.qp(Q, c, G, h)

weights = np.append(np.array(sol['x']), 1-np.sum(sol['x']))
combined_mu = np.dot(weights.reshape((-1,1)).T,mu)
Q = np.mean(alpha, axis = -1)
c = np.mean(beta, axis = -1)
x=weights.reshape((-1,1))
# print('CRPS based combine')
# print('weights:',np.round(weights,2))
# print('MAPE:',mape(combined_mu, y))
# print('CRPS:',np.dot(np.dot(x.T,Q),x) + np.dot(c.T,x))
result_for_df[0,2] = mape(combined_mu, y)
result_for_df[1,2] = np.dot(np.dot(x.T,Q),x) + np.dot(c.T,x)

num_slack = y.shape[0]
n = mu.shape[0] # number of models to combine
c = matrix(1.0, (n+num_slack,1))
c[-n:] = 0.0
G = np.zeros((2*num_slack+n, num_slack+n))
for i in range(num_slack):
    G[2*i,i] = -1.0
    G[2*i+1,i] = -1.0
    G[2*i,-n:] = mu[:,i]/y[i]
    G[2*i+1,-n:] = -mu[:,i]/y[i]
G[2*num_slack:, num_slack:] = -np.eye(n)
G = matrix(G)
h = matrix(-1.0, (2*num_slack+n,1))
h[::2] = 1.0
h[-n:] = 0.0
A = matrix(0.0, (1,num_slack+n))
A[-n:,-n:] = 1.0
b = matrix(1.0)
sol = solvers.lp(c, G, h, A, b)

combined_mu = np.dot(sol['x'][-n:].T,mu)
Q = np.mean(alpha, axis = -1)
c = np.mean(beta, axis = -1)
x=sol['x'][-n:]

# print('MAPE based combine')
# print('weights',np.round(sol['x'][-n:],2))
# print('MAPE:',mape(combined_mu, y))
# print('CRPS:', np.dot(np.dot(x.T,Q),x) + np.dot(c.T,x))
result_for_df[0,1] = mape(combined_mu, y)
result_for_df[1,1] = np.dot(np.dot(x.T,Q),x) + np.dot(c.T,x)

Q = np.mean(alpha, axis = -1)
c = np.mean(beta, axis = -1)
x=1./n * np.ones((n,1))

ave_weights = 1./n * np.ones((n,1))
combined_mu = np.dot(ave_weights.T,mu)

# print('Average model:')
# print('MAPE:',mape(combined_mu, y))
# print('CRPS:',np.dot(np.dot(x.T,Q),x) + np.dot(c.T,x))
result_for_df[0,0] = mape(combined_mu, y)
result_for_df[1,0] = np.dot(np.dot(x.T,Q),x) + np.dot(c.T,x)

model_mapes = np.zeros((n,1))
for i in range(n):
    model_mapes[i] = mape(mu[i,:], y)
#print(model_mapes)

model_crps = np.zeros((n,1))
for i in range(n):
    model_crps[i] = crps(mu[i,:], sigma[i,:], y)
#print(model_crps)

df15 = pd.DataFrame(np.round(model_mapes,2))
df15['CRPS'] = np.round(model_crps,1)
df15['MAPEw'] = np.round(sol['x'][-n:],2)
df15['CRPSw'] = np.round(weights,2)

df3 = pd.DataFrame(np.round(result_for_df,2))

     pcost       dcost       gap    pres   dres
 0: -1.9981e+00 -3.0209e+00  2e+01  5e+00  2e-16
 1: -1.8609e+00 -2.6403e+00  3e+00  5e-01  3e-16
 2: -1.4323e+00 -1.8979e+00  5e-01  2e-16  4e-16
 3: -1.5022e+00 -1.5758e+00  7e-02  1e-16  2e-16
 4: -1.5280e+00 -1.5312e+00  3e-03  1e-16  2e-16
 5: -1.5303e+00 -1.5304e+00  7e-05  2e-16  2e-16
 6: -1.5304e+00 -1.5304e+00  7e-07  3e-16  2e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres   k/t
 0:  0.0000e+00 -0.0000e+00  3e+03  1e+00  2e+00  1e+00
 1:  8.9863e+00  1.0954e+01  1e+02  1e-01  2e-01  2e+00
 2:  2.4893e+01  2.5083e+01  2e+01  3e-02  4e-02  2e-01
 3:  2.9617e+01  2.9672e+01  7e+00  8e-03  1e-02  6e-02
 4:  3.1656e+01  3.1662e+01  1e+00  1e-03  3e-03  8e-03
 5:  3.2029e+01  3.2030e+01  4e-01  4e-04  7e-04  2e-03
 6:  3.2129e+01  3.2129e+01  1e-01  1e-04  2e-04  5e-04
 7:  3.2163e+01  3.2163e+01  3e-02  3e-05  5e-05  8e-05
 8:  3.2172e+01  3.2172e+01  6e-03  7e-06  1e-05  2e-05
 9:  3.2175e+01  3.2175e+01

### Result
columns are MAPE，CRPS, weights in MAPE based method, and weights in CRPS based method<br>
rows are 15 different single models

In [13]:
df15

,0,CRPS,MAPEw,CRPSw
0,5.52,28.6,-0.00,0.00
1,5.51,28.5,-0.00,0.00
2,5.47,28.8,0.04,0.00
3,5.36,27.7,-0.00,0.00
4,5.51,28.8,-0.00,0.00
5,4.29,22.1,0.02,0.00
6,4.27,23.0,0.00,0.00
7,4.35,22.6,-0.00,0.00
8,4.15,21.5,0.15,0.19
9,4.65,23.3,0.05,0.12


First row is MAPE，second row is CRPS,<br>
Columns are Simple Average, MAPE based, and  CRPS based ensemble methods

In [15]:
df3

,0,1,2
0,4.08,3.83,3.86
1,20.99,19.78,19.64
